In [ ]:
##1. AI is includes both Machine Learning and Deep learn, its the bigger concept of making machines clever. Machine learning is a part where we train computers on data, deep learning is similar but uses big neural networks to train itself more complex stuff.

##2. Keras is an interface that runs oon tensor flow to make it easier to work with, tensor flow itself is just a tool to use when making AI models.


##3. Parameters are what models learn while training whereas hyperparamters are like instructions we set the model before we start the training like how fast it should learn or how many epochs it should run on through the data.

##4. Training = Learns the model. Validation = tweaks/tunes the data during the training phase. Test = for evaluating how good the model was after training, on unseen data (the part of the data we split at teh start that the model isn't trained on).

##5. The code is building and training a neural network. The network will have 3 layers of nodes, first 100, then 50 and 1 output. 
# Its got two activations, first Relu which only allows positive values through to the bext layer, Sigmoid turns the output into binart output (1 or 0). 
# Its got an early stopping function which mkaes the model stop if there isn't any improvement in results after 5 rounds. It splits the data into 80% and 20%, 80 for training teh data and the last 20% for validation.

##6. It should stop the model from overfitting where it basically becomes a memory model its not actually look for patterns its trying to memorize the data set for perfect score, which would make it ineffective on unseen data,.

##7. its a regularising technique where random neurons are switched off during the training process to avoid overfitting.

##8. Like we saw in the code from question 5, its stops the model once we dont see more improvements, or to stop it from running for too long.

##9. CNN's aka "Convolutional Neural Networks"

##10. it puts a filter over and image, like a grid and then detects patterns inside. They're then combined into deeper layers to try and detect more complex things like objects or shapes, then teh model predicts whats in the image.

##11. model.save("model_file.keras") saves the model to a keras file - my_model = load_model("model_file.keras") loads said model

##12. CPU is the processor in the computer, GPU is the special processor which takes care of multiple jobs simultaneously,

In [ ]:
!pip install pypdf
!pip install tf-keras
!pip install langchain_community
!pip install langchain-openai
!pip install sentence-transformers
!pip install faiss-cpu
!pip install huggingface_hub[hf_xet]

In [ ]:
##

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Loads the PDF as one doc
loader = PyPDFLoader(r"C:\Users\ahern\Desktop\Kunskap DL2\PlayerDnDBasicRules_v0.2_PrintFriendly.pdf")
data = loader.load()

# Split the text into chunks, each chunk will be 1000 characters long with an overlap of 100 characters
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(data)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# HEre we use the HuggingFaceEmbeddings to create embeddings for the text chunks
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import FAISS

# Here we create a FAISS vector store from the documents and embeddings
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
## This is the second actual model i made, which i will explain deeper later in the report. This model is using the FLan-T5 model from HuggingFace, which is a text-to-text model. It can be used for various tasks like summarization, translation, and question answering.

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline
# Load the FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Createss a text2text generation pipeline
text2text_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200
)
# makes a HuggingFacePipeline instance
llm = HuggingFacePipeline(pipeline=text2text_pipeline)

In [ ]:
from langchain.chains import RetrievalQA

# Create a RetrievalQA chain that uses the retriever and the LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  
    return_source_documents=True
)

In [ ]:
## Heres the basic use of my model, you can simply edit inside the "" after query = to ask it different questions. Or in the next cell you'll see i made a terminal loop where you can ask questions.
query = "What is AC?"
result = qa_chain({"query": query})

print(result["result"])

In [ ]:
## Here is the you can ask questions in the tyerminal to the model until you type "exit" to stop the program.

while True:
    user_input = input("Ask a question about D&D (or type 'exit'): ")
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    response = qa_chain.invoke({"query": user_input})
    print("Answer:", response["result"])

In [ ]:

## So the question in the assignment was how this code could be used i reality, and which potentual challenges and opportunities exist. Firstly I want to quickly explain why I used a document about Dungeons and Dragons (D&D) as my data source?
# I Chose the topic because its something i know like the back of my hand, which allowed me to focus on the technical aspects of the model rather than trying to understand the data. 
# It also meant that i would be able to tell straaight away if the model was working properly or not, I know the content of the document well enough to know the difference between an accurate and inaccurate answer from my model.
# I'd like to also note that the pdf i used is a free doc, 180 pages, that i sourced from the wizards of the coast website, which is the company that makes D&D. https://media.wizards.com/2018/dnd/downloads/DnD_BasicRules_2018.pdf

# So, my actual plan was to make a simple RAG model which could be used for onboarding personel at a company. From experience i have found that new staff are commonly given training documents or manueals to read on arrival, which they then have to remember and use later on.
# This might be easy for some people, but for others, myself included, it can be hard to remember everything you read in a manual. It would be nice to be able to ask questions about the document and get quick accurate answers based on what was in the writings.
# My model runs on the idea that you've read the document, and have follow up questions about its content. 
# So I would imagine this model would be intgrated into a companies internal systems the made avaulable to all employees, not just newbies.

# Now, for the challenges, well, first of all you'd have to make sure the docs or pdfs are constantly updated, for yuou could end up giving out outdated info to staff. 
# There might also be a problem with so vague questions, it might rely too much of the level of how much the person has read the documents.
# The model i used is free and easy to use, but if it were used in a real company, you might want to use a more advanced model, which could be costly.
# Honeestly speaking, the model is a little slow to answer, not terrible, but not super fast either.

## Now i wnat to abnswer, why did i chose the FLan 5 model? Well, after i tested a previous model (tiiuae/falcon-rw-1b), i found its answers were not only unhelpful, but kind of junk and hard to read
## Here is an example of the answer i got from the first model i rejected:

## " 9D&D Player’s Basic Rules v0.2 | Chapter 1: Step-by-Step Characters
## AC using the rules in chapter 5. Record your AC on 
## your character sheet.
##  Your character needs to be proficient with armor and 
## shields to wear and use them effectively, and your armor 
## and shield proficiencies are determined by your class. 
## There are drawbacks to wearing armor or carrying a 
## shield if you lack the required proficiency, as explained 
## in chapter 5.
##  Some spells and class features give you a different 
## way to calculate your AC. If you have multiple features 
## that give you different ways to calculate your AC, you 
## choose which one to use.
## Weapons
## For each weapon your character wields, calculate the 
## modifier you use when you attack with the weapon and 
## the damage you deal when you hit.
##  When you make an attack with a weapon, you roll 
## a d20 and add your proficiency bonus (but only if you 
## are proficient with the weapon) and the appropriate 
## ability modifier.
## • For attacks with melee weapons, use your Strength 
## ...
## and straps distribute the weight over the body.

## Question: What is AC?
## Helpful Answer:!koochum;,and ICD,.Andac;moo;,IACI,theclat;·moac;IAC;Now,the.,ThecorrectThe;,corrected,,and,, and.,,,,IACI,I,ICorrected.,IACI,,I,corrected,.,,,.,.,,IACI,I,,,,And,,,.,.,DO,,,I,,,.,I,,,,,,,I,,,I,I,,I,I,,EdI,I,,,.,II,,,,,DO,.,,,.,,,and,,,,,,DO,,I,�,.,,,,,,,,,,,,,,,,,,,(,,,
## Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...""



## As you can see, the model is not very good at answering the question, it just gives a lot of random text that doesn't really answer the question. Lets compare that to the answer i got from the current model:

## Armor Class Your Armor Class (AC) represents how well your character avoids being wounded in battle. Things that contribute to your AC include the armor you wear, the shield you carry, and your Dexterity modifier. 
# Without armor or a shield, your AC equals 10 + his or her Dexterity modifier. Without armor or a shield, your AC equals 10 + his or her Dexterity modifier